In [1]:
#!pip install PyPDF2

In [12]:
import PyPDF2
from pathlib import Path

def extract_text_from_pdf(pdf_file):
    pdf_path = Path(pdf_file)
    
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)  # Utilise PdfReader
        text = ""
        for page in reader.pages[5:]:
            text += page.extract_text()
        return text
text = extract_text_from_pdf(r"D:\Generative IA\rag_pdfs\pdf_data\M98232301-03.pdf")
#print(text)


In [13]:
import re
def extract_titles_and_content(text):
# title_pattern = r"((?:\d+\.)*\d+(?:\.\d+)*[-.]?\s*[^\n]+)(.*?)(?=\n(?:\d+\.)*\d+(?:\.\d+)*[-.]?\s*[^\n]+|\Z)"
    title_pattern = r"((?:\d{1,2}\.)+\d{1,2}[-]?\s*[^\n]+|(?<!\d)(?=.*[A-Za-z])[^\n]+?)(.*?)(?=\n(?:\d{1,2}\.)+\d{1,2}[-]?\s*[^\n]+|\Z)"



    # Utilisation de findall pour capturer toutes les occurrences des titres
    titles = re.findall(title_pattern, text, re.DOTALL)
    cleaned_titles = [value for tup in titles for value in tup if value.strip() and len(value)>5]
    #print(cleaned_titles)
    results = []
    for match in titles:
        title = match[0].strip()  # Titre avec le numéro
        content = match[1].strip()  # Contenu jusqu'au prochain titre
        if len(title)<70 and len(content.strip().replace('"', '')) > 5 and "Manual" not in title and "Only in SCADA and Deluxe versions  PowerStudio" not in title  and "Barcelona" not in title:
    # Votre code ici
            
          #  print("TITLE: ",title)
          #  print("Content",content)
          #  print("########################################################\n")
            results.append((title, content))
    
    return results


In [14]:
title_content_paires=extract_titles_and_content(text)
for a,b in title_content_paires:
    print(a)
    print("#############")
    
#print(title_content_paires[3])


#############
1.1.- POWERSTUDIO TOPOLOGIES
#############
1.1.1  All in one
#############
1.1.2  Engine /editor and clients
#############
1.1.3  Engine/editor and clients
#############
1.1.4  Engine, editors and clients
#############
1.2.- POWERSTUDIO I NSTALLATION
#############
1.3.- TYPICAL  APPLICATION
#############
1.3.1  Configure the communications engine
#############
1.3.2  Create the application in the editor
#############
1.3.3  Export the application to the engi ne
#############
1.3.4  View the application in the client
#############
3.1.- XML Services
#############
3.1.1  / services / user / devices.xml
#############
3.1.2  /services/user/deviceInfo.xml?id=device?...
#############
3.1.5  / services / user / forceVariables.xml? id = device
#############
3.1.6  /services/user/records.xml?begin=…?end=…?var=…?period=900
#############
3.1.7  /services/user/events.xml?begin=…?end=…?id=…
#############
3.1.8  /services/user/recordsEve.xml?begin=…?end=…?id=…
#############
3.2.1  Som

In [4]:
#!pip install spacy transformers nltk
#!python -m spacy download en_core_web_sm


In [5]:
def remove_stopwords(text):
    doc = nlp(text)
    # Exclure les mots qui sont des stop words
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    return ' '.join(filtered_tokens)


In [6]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline

# Charger le modèle spaCy
nlp = spacy.load('en_core_web_sm')

# Charger le modèle Llama 3 pour la génération de texte sur GPU
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B",device=0)     
# Génération des questions
def generate_questions(title):
    prompt = (
        "Generate a relevant question based on the title related to Circuttor Power Studio SCADA.\n\n"
        "Title: 'ACCESS VIA THE POWERSTUDIO CLIENT'\n"
        "Question: 'How can I access the PowerStudio SCADA client?'\n\n"
        "Title: 'MONITORING POWER CONSUMPTION'\n"
        "Question: 'What tools are available for monitoring power consumption in PowerStudio SCADA?'\n\n"
        "Title: 'CONFIGURATION OF POWER DEVICES'\n"
        "Question: 'How do I configure power devices in PowerStudio SCADA?'\n\n"
        f"Title: '{title}'\n"
        "Question:"
    )


    # Génération de texte
    questions = pipe(prompt, max_new_tokens=50, num_return_sequences=1)
    return questions[0]['generated_text'].split(f"Title: '{title}'\n")[1].split(":")[1]

# Suppression des stop words
def remove_stopwords(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if not token.is_stop])

# Nettoyage basique
def clean_text(text):
    return text.replace('\n', ' ').strip()


def clean_title(title):
    # Supprimer les formats comme X.-, X.X-, X.X.X et X.X.X.
    cleaned_title = re.sub(r'^[\d]+[-.]*[\s]*', '', title).strip()
    cleaned_title = re.sub(r'(\d+[\.\-])', '', cleaned_title).strip() 
    return cleaned_title

# Pipeline de préprocessing
def preprocess_pipeline(title, content):
    # 1. Nettoyage basique
    content = clean_text(content)
    
    # 2. Suppression des stop words
    content = remove_stopwords(content)
    
    # 3. Tokenisation et Lemmatisation
    doc = nlp(content)
    lemmatized = ' '.join([token.lemma_ for token in doc])
    
    # 4. Vectorisation (TF-IDF)
    vectorizer = TfidfVectorizer()
    vectorized_text = vectorizer.fit_transform([lemmatized])
    title=clean_title(title)
    # 5. Génération de questions à partir du titre et du contenu
    question = generate_questions(title)
    
    return {
       # "lemmatized": lemmatized,
       # "vectorized_text": vectorized_text,
        "question": question,
        "content":content
    }
import json
# Application du pipeline de prétraitement à chaque contenu extrait
list_results = []
with open('D:\\Generative IA\\rag_pdfs\\manuel_3_QA.json', 'w', encoding='utf-8') as f:
    for title, content in title_content_paires:
        print(f"Title: {title}")
        result = preprocess_pipeline(title, content)
        list_results.append(result)
        
        # Remplissage du fichier JSON
        json.dump(result, f, ensure_ascii=False)
        f.write("\n")  # Pour séparer chaque entrée JSON sur une nouvelle ligne

        for key, value in result.items():
            print(f"{key}: {value}")
        print("\n########################################\n")


Title: 


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
C:\Users\simo_\anaconda3\lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


question:  ''

Title
content: PowerStudio                                                                                                            PowerSt udio       6                                                                                                                                     User Manual    LOG REVISIONS       Date   Revisio n Description    11/13   M98232301 -03 - 13A Original version    04/16 M98232301- 03 - 16A Modification section :    2 .   05/16 M98232301- 03 - 16B Modification section :    2 .   09/16 M98232301- 03 - 16C Modification section :    2 .   08/18   M98232301 -03 - 18A versi changed 4.0.10                                                                                                     PowerStudio              User Manual                                                                                                                                     7                                                                                          

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


question:  'What is a PowerStudio topology?'

Title
content: PowerStudio software design -differentiated modules : editor , engine   cli ent , possible use different topologies , explained detail :

########################################

Title: 1.1.1  All in one


...

question:  'How can I display documents from a SCADA screen?'


content: documents previously stored PC run control . wish   PDF file , enter directly ( including complete path ) program field ,   file open related , defined program running screen clicking   control .   Likewise , file type ass ociated program open . ( DOC ,   TXT , HTML , WAV , MP3 , MPG , AVI , etc . )       N.B : option work systems run Windows ,   applet .

########################################

Title: 3.2.23 How can I obtain an event according to the status of a device?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


question:  'How can I obtain an event according to the status of a device?'

Title
content: produce event ac cording status devices , use STATUS variable   device event activation status . possible values STATUS variable seen   variable type appendix .       events generate number variables associated ,   times event enabled , times deactivated , recognized   long enabled .

########################################

Title: 3.2.24 How can I produce sounds in response to an event?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


question:  'How can I produce sounds in response to an event?'


content: produce sound response event ,   add action run external   program enter corresponding ( WAV , MP3 , etc . ) path program field .   action effect , need program installed play type file (   example , Windows Media Player , Sonique , Winamp , etc . ) .     needed simply reproducing bleep client application , action associated   event added carry . Likewise action occur activation ,   acknow ledgement , deactivation active .        Support audible alarm action client response event       PowerStudio                                                                                                            PowerSt udio       40                                                                                                                                     User Manual

########################################

Title: 3.2.25 How do I show documents in response to an event?
question:  'How can I show documents in resp

In [11]:
print(list_results[5])

{'question': " 'How to use the editor and clients of PowerStudio SCADA?'\n\nTitle", 'content': 'configuration editor engine machine   clients machines connect engine access viewing data , SCADA screens ,   reports , etc . configuration useful want view engine data remotely client .        PowerStudio                                                                                                            PowerSt udio       8                                                                                                                                     User Manual'}


In [ ]:
from transformers import pipeline

# Utilisation de DistilGPT-2 pour la génération de texte
pipe = pipeline("text-generation", model="distilgpt2", device=0)

def generate_question(title):
    prompt = f"Based on the title: '{title}', generate a relevant question specifically related to Circuttor Power Studio SCADA."
    
    question = pipe(prompt, max_new_tokens=50)
    
    # Extraire la question générée
    return question[0]["generated_text"].split(":")[1].strip()  


question=generate_question(a)

('1.- ACCESS VIA THE POWERSTUDIO CLIENT',
 "The software has a client application that will allow users to acces s SCADA screens, reports \nand to display instantaneous values being measured by devices either locally or through a remote \nconnection. Graphs of the values recorded by the devices can also be prepared and listings accessed, \nevents can be viewed, device stat us can be displayed, etc. Similarly, this client is available as an \nembedded applet on the application's webpage.")